Imports

In [38]:
import numpy as np
from sklearn.linear_model import LassoCV, RANSACRegressor, Lasso, TheilSenRegressor
from sklearn.model_selection import cross_val_score, train_test_split
import matplotlib.pyplot as plt

In [39]:
def sum_squared_errors(estimator, X, y):
    return np.sum((np.reshape(estimator.predict(X), (y.shape)) - y) ** 2)

In [40]:
def mean_squared_errors(estimator, X, y):
    return np.mean((np.reshape(estimator.predict(X), (y.shape)) - y) ** 2)

Load data

In [41]:
data_x = np.load('Xtrain_Regression2.npy')
data_y = np.load('Ytrain_Regression2.npy')

Fit a linear regression model using all the data

In [42]:
folds = 10

lasso = LassoCV(cv=folds).fit(data_x, data_y.ravel())

print(lasso.alpha_)
print(lasso.score(data_x, data_y))

0.17950930279330035
0.20824558947660365


### Finding outliers

Z values

In [43]:
x_means = data_x.mean(axis=0)
x_stds = data_x.std(axis=0)
y_mean = data_y.mean(axis=0)
y_std = data_y.std(axis=0)

x_z_scores = (data_x - x_means) / x_stds
y_z_scores = (data_y - y_mean) / y_std

x_outliers = data_x[np.abs(x_z_scores) > 2]
y_outliers = data_y[np.abs(y_z_scores) > 2]

In [44]:
vec=np.arange(0.5, 0.9, 0.001) 
best_cv_score=0
for k in vec:
    ransac = RANSACRegressor(residual_threshold= k, random_state=0).fit(data_x, data_y.ravel()) #tests the estimator with different thresholds
    cv_score = cross_val_score(ransac, data_x[ransac.inlier_mask_], data_y.ravel()[ransac.inlier_mask_], cv=folds, scoring="neg_mean_squared_error").mean() #takes the MSE into account with the CV excluding the outliers
    if(best_cv_score<cv_score) and ransac.inlier_mask_.sum()>78: #keeps the best cv score and makes sure at least 79 inliers are inclued
        best_k=k

ransac = RANSACRegressor(residual_threshold= best_k, random_state=0).fit(data_x, data_y.ravel()) #final estimator with the optimized value


#cv_score = cross_val_score(ransac, data_x[ransac.inlier_mask_], data_y.ravel()[ransac.inlier_mask_], cv=folds)
   
#sse=sum_squared_errors(ransac, data_x[ransac.inlier_mask_], data_y.ravel()[ransac.inlier_mask_])
#model = LassoCV(cv=folds, alphas=np.arange(0.001, 1, 0.001)).fit(data_x[ransac.inlier_mask_], data_y.ravel()[ransac.inlier_mask_])

#print(model.alpha_)
#print(cross_val_score(model, data_x[ransac.inlier_mask_], data_y.ravel()[ransac.inlier_mask_], cv=folds).mean())
#print(sum_squared_errors(model, data_x[ransac.inlier_mask_], data_y.ravel()[ransac.inlier_mask_]))


#print(sum_squared_errors(ransac, data_x[ransac.inlier_mask_], data_y.ravel()[ransac.inlier_mask_]))

#for feature in range(10):
    #plt.figure(feature)
   # plt.scatter(data_x[:,feature], data_y)  
    #plt.scatter(data_x[ransac.inlier_mask_, feature], data_y[ransac.inlier_mask_], color='red')
    #plt.plot(data_x[:, feature], data_x[:, feature] * ransac.estimator_.coef_[feature], color='red')
   #plt.plot(data_x[:, feature], data_x[:, feature] * lasso.coef_[feature], color='green')
  #  plt.show()

0.9964749194384579
best_sse=0.008683
best_k=0.546000


In [45]:
theilsen = TheilSenRegressor(max_subpopulation=90).fit(data_x, data_y.ravel())
cv_score = cross_val_score(theilsen, data_x[ransac.inlier_mask_], data_y.ravel()[ransac.inlier_mask_], cv=folds)
print(cv_score.mean()) #scores negativos (?)


0.9963582182366052
